In [1]:
import time
import datetime
# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

start_time = time.time()
print("  Training epoch took: {:}".format(format_time(time.time() - start_time)))

  Training epoch took: 0:00:00


In [2]:
"""Training IGMC model on the MovieLens dataset."""

import os
import sys
import time
import glob
import random
import argparse
from shutil import copy

import numpy as np
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from explicit_model_rotten import IGMC
from explicit_data_rotten import RottenTomato
from explicit_dataset_rotten import (RottenTomatoDataset, collate_rotten_tomato,
                                     MultiRottenTomatoDataset, multi_collate_rotten_tomato)
from utils import MetricLogger

Using backend: pytorch


In [3]:
def evaluate(model, loader, device):
    predict_list = list()
    label_list = list()
    
    # Evaluate RMSE
    model.eval()
    mse = 0.
    for iter_idx, batch in enumerate(loader):
        with th.no_grad():
            block_r = batch[0][0].to(device)
            block_s = batch[0][1].to(device)
            block_e = batch[0][2].to(device)
            tmp = model(block_r, block_s, block_e)
            preds = (tmp + 1)/2
        
        tmp_rating = batch[1][0].to(device) # 정답 rating labels
        labels = (tmp_rating+1)/2
        mse += ((preds - labels) ** 2).sum().item()
        
        predict_list.append(preds.tolist()) # 예측값 저장
        label_list.append(labels.tolist()) # 정답값 저장
        
        # 2차원 -> 1차원 리스트 변형
    predict_list = [element for array in predict_list for element in array]
    label_list = [element for array in label_list for element in array]    
        
    mse /= len(loader.dataset)
    rmse = np.sqrt(mse)
    return rmse, predict_list, label_list

def adj_rating_reg_r(model):
    arr_loss = 0
    for conv in model.convs_r:
        weight = conv.weight.view(conv.num_bases, conv.in_feat * conv.out_feat)
        weight = th.matmul(conv.w_comp, weight).view(conv.num_rels, conv.in_feat, conv.out_feat)
        arr_loss += th.sum((weight[1:, :, :] - weight[:-1, :, :])**2)
    return arr_loss

def adj_rating_reg_s(model):
    arr_loss = 0
    for conv in model.convs_s:
        weight = conv.weight.view(conv.num_bases, conv.in_feat * conv.out_feat)
        weight = th.matmul(conv.w_comp, weight).view(conv.num_rels, conv.in_feat, conv.out_feat)
        arr_loss += th.sum((weight[1:, :, :] - weight[:-1, :, :])**2)
    return arr_loss

# rating, sentiment, emotion loader를 받음
def train_epoch(model, loss_fn, optimizer, arr_lambda, loader, device, log_interval):
    model.train()

    epoch_loss = 0.
    iter_loss = 0.
    iter_mse = 0.
    iter_cnt = 0
    iter_dur = []

    iter_idx = 0
    
    # 저장 리스트(예측, 정답)
    predict_list = list()
    label_list = list()
    
    for iter_idx, batch in enumerate(loader, start=1):
        t_start = time.time()
        
        # rating
        inputs_r = batch[0][0].to(device)
        labels_r = batch[1][0].to(device)
        # sentiment
        inputs_s = batch[0][1].to(device)
        labels_s = batch[1][1].to(device)
        # emotion
        inputs_e = batch[0][2].to(device)
        labels_e = batch[1][2].to(device)
    
        preds = model(inputs_r, inputs_s, inputs_e)
        preds  = (preds + 1)/2
        labels_r = (labels_r + 1)/2
        loss = loss_fn(preds, labels_r).mean() + arr_lambda * adj_rating_reg_r(model) + arr_lambda * adj_rating_reg_s(model)
#         loss = loss_fn(preds, labels_r).mean()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * preds.shape[0]
        iter_loss += loss.item() * preds.shape[0]
        iter_mse += ((preds - labels_r) ** 2).sum().item()
        iter_cnt += preds.shape[0]
        iter_dur.append(time.time() - t_start)

        predict_list.append(preds.tolist()) # 예측값 저장
        label_list.append(labels_r.tolist()) # 정답값 저장
        
        if iter_idx % log_interval == 0:
            print("Iter={}, loss={:.4f}, mse={:.4f}, time={:.4f}".format(
                iter_idx, iter_loss/iter_cnt, iter_mse/iter_cnt, np.average(iter_dur)))
            iter_loss = 0.
            iter_mse = 0.
            iter_cnt = 0
        
    # 2차원 -> 1차원 리스트 변형
    predict_list = [element for array in predict_list for element in array]
    label_list = [element for array in label_list for element in array]

    train_epoch_loss = epoch_loss / len(loader.dataset)  
    return train_epoch_loss, predict_list, label_list

## 1. Config

In [4]:
import easydict

args = easydict.EasyDict({ 
    'data_name':            'rotten',
    'testing':     	        True,
    'device':      	        0,
    'seed':        	        1234,
    'data_test_ratio':      0.1,
    'num_workers':   	    8,
    'data_valid_ratio':     0.2,
    'train_log_interval':   200,
    'valid_log_interval':   10,
    'save_appendix':   	    'debug',
    'hop':   	            1,
    'sample_ratio':    	    1.0,
    'max_nodes_per_hop':    100,
    'edge_dropout':   	    0.2,
    'force_undirected':     False,
    'train_lr':   	        1e-3,
    'train_min_lr':   	    1e-6,
    'train_lr_decay_factor':0.1,
    'train_lr_decay_step':  50,
    'train_epochs':   	    10,
    'batch_size':   	    16,
    'arr_lambda':   	    0.001,
    'num_rgcn_bases':   	4,
    'train_epochs':   	    1
})

In [5]:
### set save_dir according to localtime and test mode
file_dir = os.path.dirname(os.path.realpath('__file__'))
val_test_appendix = 'testmode' if args.testing else 'valmode'
local_time = time.strftime('%y%m%d%H%M', time.localtime())
args.save_dir = os.path.join(
    file_dir, 'log/{}_{}_{}_{}'.format(
        args.data_name, args.save_appendix, val_test_appendix, local_time
    )
)
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir) 
print(args)

# backup current .py files
for f in glob.glob(r"*.py"):
    copy(f, args.save_dir)

# save command line input
cmd_input = 'python3 ' + ' '.join(sys.argv)
with open(os.path.join(args.save_dir, 'cmd_input.txt'), 'a') as f:
    f.write(cmd_input)
    f.write("\n")
print('Command line input: ' + cmd_input + ' is saved.')

{'data_name': 'rotten', 'testing': True, 'device': 0, 'seed': 1234, 'data_test_ratio': 0.1, 'num_workers': 8, 'data_valid_ratio': 0.2, 'train_log_interval': 200, 'valid_log_interval': 10, 'save_appendix': 'debug', 'hop': 1, 'sample_ratio': 1.0, 'max_nodes_per_hop': 100, 'edge_dropout': 0.2, 'force_undirected': False, 'train_lr': 0.001, 'train_min_lr': 1e-06, 'train_lr_decay_factor': 0.1, 'train_lr_decay_step': 50, 'train_epochs': 1, 'batch_size': 16, 'arr_lambda': 0.001, 'num_rgcn_bases': 4, 'save_dir': 'C:\\Users\\user\\Jupyter_project\\keejun\\IGMC_CX\\log/rotten_debug_testmode_2111211722'}
Command line input: python3 C:\Users\user\anaconda3\envs\graph\lib\site-packages\ipykernel_launcher.py -f C:\Users\user\AppData\Roaming\jupyter\runtime\kernel-2aa2e963-68cd-4b84-8169-2fdecc07b3a6.json is saved.


In [6]:
args

{'data_name': 'rotten',
 'testing': True,
 'device': 0,
 'seed': 1234,
 'data_test_ratio': 0.1,
 'num_workers': 8,
 'data_valid_ratio': 0.2,
 'train_log_interval': 200,
 'valid_log_interval': 10,
 'save_appendix': 'debug',
 'hop': 1,
 'sample_ratio': 1.0,
 'max_nodes_per_hop': 100,
 'edge_dropout': 0.2,
 'force_undirected': False,
 'train_lr': 0.001,
 'train_min_lr': 1e-06,
 'train_lr_decay_factor': 0.1,
 'train_lr_decay_step': 50,
 'train_epochs': 1,
 'batch_size': 16,
 'arr_lambda': 0.001,
 'num_rgcn_bases': 4,
 'save_dir': 'C:\\Users\\user\\Jupyter_project\\keejun\\IGMC_CX\\log/rotten_debug_testmode_2111211722'}

In [7]:
random.seed(args.seed)
np.random.seed(args.seed)
th.manual_seed(args.seed)
if th.cuda.is_available():
    th.cuda.manual_seed_all(args.seed)    

## 2. Train

In [8]:
### prepare data and set model
path = './raw_data/rotten_tomato/'
rotten_tomato_r = RottenTomato('rating',    path, testing=args.testing,test_ratio=args.data_test_ratio, valid_ratio=args.data_valid_ratio)
rotten_tomato_s = RottenTomato('sentiment', path, testing=args.testing,test_ratio=args.data_test_ratio, valid_ratio=args.data_valid_ratio)
rotten_tomato_e = RottenTomato('emotion',   path, testing=args.testing,test_ratio=args.data_test_ratio, valid_ratio=args.data_valid_ratio)

Label_type: rating
	Train rating pairs : 216328
	Valid rating pairs : 43266
	Test rating pairs  : 28766
Label_type: sentiment
	Train rating pairs : 216328
	Valid rating pairs : 43266
	Test rating pairs  : 28766
Label_type: emotion
	Train rating pairs : 216328
	Valid rating pairs : 43266
	Test rating pairs  : 28766


### 2-1. multi_rotten_tomato_dataset 정의

In [9]:
# 수정한 것 (단, 그래프의 모든 행의 길이 동일함)
train_rating_pairs  = [rotten_tomato_r.train_rating_pairs, rotten_tomato_s.train_rating_pairs, rotten_tomato_e.train_rating_pairs]
train_rating_values = [rotten_tomato_r.train_rating_values, rotten_tomato_s.train_rating_values, rotten_tomato_e.train_rating_values]
train_graph         = [rotten_tomato_r.train_graph, rotten_tomato_s.train_graph, rotten_tomato_e.train_graph]

test_rating_pairs  = [rotten_tomato_r.test_rating_pairs, rotten_tomato_s.test_rating_pairs, rotten_tomato_e.test_rating_pairs]
test_rating_values = [rotten_tomato_r.test_rating_values, rotten_tomato_s.test_rating_values, rotten_tomato_e.test_rating_values]
test_graph         = [rotten_tomato_r.train_graph, rotten_tomato_s.train_graph, rotten_tomato_e.train_graph]

valid_rating_pairs  = [rotten_tomato_r.valid_rating_pairs, rotten_tomato_s.valid_rating_pairs, rotten_tomato_e.valid_rating_pairs]
valid_rating_values = [rotten_tomato_r.valid_rating_values, rotten_tomato_s.valid_rating_values, rotten_tomato_e.valid_rating_values]
valid_graph         = [rotten_tomato_r.train_graph, rotten_tomato_s.train_graph, rotten_tomato_e.train_graph]

hop = [1, 1, 1]
sample_ratio = [1.0, 1.0, 1.0]
max_nodes_per_hop = [200, 200, 200]

In [10]:
train_dataset = MultiRottenTomatoDataset(train_rating_pairs, train_rating_values, train_graph, hop, sample_ratio, max_nodes_per_hop)
train_loader = th.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, 
                        num_workers=0, collate_fn=multi_collate_rotten_tomato)

In [11]:
if args.testing:
    print('testing')
    test_dataset = MultiRottenTomatoDataset(test_rating_pairs, test_rating_values, test_graph, hop, sample_ratio, max_nodes_per_hop)
    test_loader = th.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, 
                            num_workers=0, collate_fn=multi_collate_rotten_tomato)
else:
    print('valid')
    test_dataset = MultiRottenTomatoDataset(valid_rating_pairs, valid_rating_values, valid_graph, hop, sample_ratio, max_nodes_per_hop)
    test_loader = th.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=True, 
                            num_workers=0, collate_fn=multi_collate_rotten_tomato)

testing


테스트중입니다

In [12]:
for iter_idx, batch in enumerate(train_loader, start=1):
    if iter_idx == 5:
        break
    print(iter_idx)
    
    device = args.device
    
    # rating
    inputs_r = batch[0][0].to(device)
    labels_r = batch[1][0].to(device)
    # sentiment
    inputs_s = batch[0][1].to(device)
    labels_s = batch[1][1].to(device)
    # emotion
    inputs_e = batch[0][2].to(device)
    labels_e = batch[1][2].to(device)

1
2
3
4


In [13]:
in_feats = (args.hop+1)*2 #+ rotten_tomato.train_graph.ndata['refex'].shape[1]
model = IGMC(in_feats=in_feats, 
             latent_dim=[32, 32, 32, 32],
             num_relations=10, # rotten_tomato.num_rating, 
             num_bases=4, 
             regression=True, 
             edge_dropout=args.edge_dropout,
        ).to(args.device)
loss_fn = nn.MSELoss().to(args.device)
optimizer = optim.Adam(model.parameters(), lr=args.train_lr, weight_decay=0)
print("Loading network finished ...\n")

Loading network finished ...



In [ ]:
### prepare the logger
logger = MetricLogger(args.save_dir, args.valid_log_interval)

best_epoch = 0
best_rmse = np.inf
### declare the loss information
print("Start training ...")

start_time = time.time()

# 마지막 epoch의 결과를 저장함.
predict_train_list = list()
label_train_list = list()

predict_valid_list = list()
label_valid_list = list()
best_predict_valid_list = list()
best_label_valid_list = list()

predict_test_list = list()
label_test_list = list()
best_predict_test_list = list()
best_label_test_list = list()

for epoch_idx in range(1, 20):
    print ('Epoch', epoch_idx)

    train_loss, predict_train_list, label_train_list  = train_epoch(model, loss_fn, optimizer, args.arr_lambda, 
                            train_loader, args.device, args.train_log_interval)
    test_rmse, predict_test_list, label_test_list = evaluate(model, test_loader, args.device)
    eval_info = {
        'epoch': epoch_idx,
        'train_loss': train_loss,
        'test_rmse': test_rmse,
    }
    print('=== Epoch {}, train loss {:.6f}, test rmse {:.6f} ==='.format(*eval_info.values()))

    if epoch_idx % args.train_lr_decay_step == 0:
        for param in optimizer.param_groups:
            param['lr'] = args.train_lr_decay_factor * param['lr']

    logger.log(eval_info, model, optimizer)
    if best_rmse > test_rmse:
        best_rmse = test_rmse
        best_epoch = epoch_idx
        
        best_predict_test_list = predict_test_list 
        best_label_test_list = label_test_list

    print("  Training epoch took: {:}".format(format_time(time.time() - start_time)))

Start training ...
Epoch 1
Iter=200, loss=0.6880, mse=0.6861, time=0.1299
Iter=400, loss=0.6836, mse=0.6816, time=0.1343
Iter=600, loss=0.6856, mse=0.6836, time=0.1362
Iter=800, loss=0.6757, mse=0.6737, time=0.1493
Iter=1000, loss=0.6799, mse=0.6779, time=0.1537
Iter=1200, loss=0.6764, mse=0.6744, time=0.1608
Iter=1400, loss=0.6864, mse=0.6843, time=0.1712
Iter=1600, loss=0.6792, mse=0.6771, time=0.1795
Iter=1800, loss=0.6594, mse=0.6572, time=0.1855
Iter=2000, loss=0.6572, mse=0.6549, time=0.1907
Iter=2200, loss=0.6624, mse=0.6602, time=0.1948
Iter=2400, loss=0.6977, mse=0.6955, time=0.1982
Iter=2600, loss=0.6670, mse=0.6648, time=0.2012
Iter=2800, loss=0.6732, mse=0.6710, time=0.2035
Iter=3000, loss=0.6472, mse=0.6450, time=0.2058
Iter=3200, loss=0.6767, mse=0.6745, time=0.2074
Iter=3400, loss=0.6704, mse=0.6683, time=0.2029
Iter=3600, loss=0.6643, mse=0.6622, time=0.1981
Iter=3800, loss=0.6794, mse=0.6773, time=0.1939
Iter=4000, loss=0.6748, mse=0.6728, time=0.1901
Iter=4200, loss=0

In [15]:
best_rmse

0.8246579296371273

In [53]:
import pandas as pd
train_emotion_df = pd.DataFrame([x for x in zip(predict_train_list, label_train_list)])
train_emotion_df.rename(columns={0:'predict', 1:'label'}, inplace=True)

# valid_emotion_df = pd.DataFrame([x for x in zip(predict_valid_list, label_valid_list)])
# valid_emotion_df.rename(columns={0:'predict', 1:'label'}, inplace=True)

test_emotion_df = pd.DataFrame([x for x in zip(best_predict_test_list, best_label_test_list)])
test_emotion_df.rename(columns={0:'predict', 1:'label'}, inplace=True)

NameError: name 'predict_train_list' is not defined

In [99]:
path = './raw_data/rotten_tomato/explicit/'
rate = 1
train_emotion_df.to_csv(path + f'explicit_train_emotion_{rate}.csv', index=False)
# valid_emotion_df.to_csv(path + f'explicit_valid_emotion_{rate}.csv', index=False)
test_emotion_df.to_csv(path + f'explicit_test_emotion_{rate}.csv', index=False)

Training ends. The best testing rmse is 0.804960 at epoch 2


### Train_epoch 함수 테스트

In [25]:
model = model
loss_fn = loss_fn
optimizer = optimizer
arr_lambda = args.arr_lambda
loader = train_loader
device = args.device
log_interval = args.train_log_interval

In [26]:
log_interval

200

In [34]:
for iter_idx, batch in enumerate(loader, start=1):
    print(iter_idx)
    if iter_idx == 10:
        break

1
2
3
4
5
6
7
8
9
10


In [27]:
start_time = time.time()

model.train()

epoch_loss = 0.
iter_loss = 0.
iter_mse = 0.
iter_cnt = 0
iter_dur = []

# 서브그래프 단위로 학습
for iter_idx, batch in enumerate(loader, start=1):
    t_start = time.time()

    inputs = batch[0].to(device)
    labels = batch[1].to(device)
    preds = model(inputs)
    loss = loss_fn(preds, labels).mean() + arr_lambda * adj_rating_reg(model)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item() * preds.shape[0]
    iter_loss += loss.item() * preds.shape[0]
    iter_mse += ((preds - labels) ** 2).sum().item()
    iter_cnt += preds.shape[0]
    iter_dur.append(time.time() - t_start)

    if iter_idx % log_interval == 0:
        print("Iter={}, loss={:.4f}, mse={:.4f}, time={:.4f}".format(
            iter_idx, iter_loss/iter_cnt, iter_mse/iter_cnt, np.average(iter_dur)))
        iter_loss = 0.
        iter_mse = 0.
        iter_cnt = 0

train_epoch_loss = epoch_loss / len(loader.dataset)

print("  Time took: {:}".format(format_time(time.time() - start_time)))

Iter=200, loss=5.0480, mse=5.0403, time=0.0473
Iter=400, loss=3.2373, mse=3.2299, time=0.0425
Iter=600, loss=3.2133, mse=3.2064, time=0.0419
Iter=800, loss=3.1650, mse=3.1585, time=0.0415
Iter=1000, loss=3.1144, mse=3.1082, time=0.0413
Iter=1200, loss=3.0636, mse=3.0578, time=0.0411
Iter=1400, loss=2.9595, mse=2.9539, time=0.0410
Iter=1600, loss=2.9292, mse=2.9239, time=0.0408
Iter=1800, loss=2.8808, mse=2.8756, time=0.0408
Iter=2000, loss=2.9187, mse=2.9137, time=0.0409
Iter=2200, loss=2.9908, mse=2.9861, time=0.0410
Iter=2400, loss=2.9135, mse=2.9088, time=0.0412
Iter=2600, loss=2.8839, mse=2.8795, time=0.0411
Iter=2800, loss=2.9526, mse=2.9484, time=0.0411
Iter=3000, loss=2.7681, mse=2.7641, time=0.0411
Iter=3200, loss=2.9428, mse=2.9389, time=0.0410
Iter=3400, loss=2.8883, mse=2.8843, time=0.0410
Iter=3600, loss=2.8257, mse=2.8217, time=0.0413
Iter=3800, loss=2.8143, mse=2.8105, time=0.0414
Iter=4000, loss=2.8891, mse=2.8853, time=0.0417
Iter=4200, loss=2.7421, mse=2.7384, time=0.0

In [31]:
model

IGMC

In [28]:
inputs

Graph(num_nodes=1120, num_edges=31588,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'nlabel': Scheme(shape=(4,), dtype=torch.float32), 'x': Scheme(shape=(4,), dtype=torch.float32)}
      edata_schemes={'etype': Scheme(shape=(), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64), 'edge_mask': Scheme(shape=(), dtype=torch.float32)})

In [32]:
preds

tensor([3.0763, 7.5723, 4.1323, 5.4378, 5.5795, 6.2285, 6.7028, 4.2875],
       device='cuda:0', grad_fn=<MulBackward0>)

In [29]:
labels

tensor([6., 7., 6., 7., 7., 7., 9., 5.], device='cuda:0')

In [30]:
loss

tensor(2.9040, device='cuda:0', grad_fn=<AddBackward0>)

In [29]:
preds.shape[0]

32

In [30]:
train_epoch_loss

2.727407252259081

In [31]:
preds

tensor([7.0923, 5.4808, 6.1765, 8.1413, 7.6566, 4.6887, 6.3527, 6.4477, 6.0566,
        5.4327, 5.4816, 7.4522, 5.6513, 8.6706, 5.8030, 7.2858, 4.5068, 8.8621,
        3.4885, 7.3030, 3.4780, 6.1335, 2.9561, 4.1378, 5.3948, 6.8066, 5.6749,
        5.6852, 4.4658, 7.3438, 9.4554, 7.2134], device='cuda:0',
       grad_fn=<MulBackward0>)

### Evaluate 함수 테스트

In [43]:
model = model
loader = test_loader
device = args.device

In [63]:
start_time = time.time()
predict_ratings = list()
real_ratings = list()

# Evaluate RMSE
model.eval()
mse = 0.
for batch in loader:
    with th.no_grad():
        preds = (model(batch[0].to(device)) + 1)/ 2
    labels = (batch[1].to(device) + 1)/ 2
    mse += ((preds - labels) ** 2).sum().item()
    
    real_ratings.append(labels)
    predict_ratings.append(preds)
    
mse /= len(loader.dataset)
rmse = np.sqrt(mse)

print("  Time took: {:}".format(format_time(time.time() - start_time)))

  Time took: 0:00:27


In [64]:
preds

tensor([2.9968, 3.1041, 2.8940, 3.9403, 4.1405, 3.1857, 3.4408, 4.0894, 2.6228,
        3.2692, 2.8797, 2.1028, 2.8179, 3.5234, 3.3022, 4.0670, 4.0670, 2.9974,
        2.1890, 2.4611, 1.2208, 2.3978, 3.7354, 2.5466, 3.1223, 3.4322, 2.1431,
        2.5878, 2.5878, 3.6192], device='cuda:0')

In [65]:
labels

tensor([3.5000, 3.5000, 3.5000, 5.0000, 4.0000, 4.5000, 4.5000, 4.0000, 4.0000,
        3.5000, 4.5000, 1.0000, 3.0000, 2.5000, 4.0000, 2.0000, 2.0000, 4.0000,
        2.5000, 2.5000, 2.5000, 2.5000, 5.0000, 3.0000, 3.5000, 5.0000, 3.5000,
        3.0000, 3.0000, 4.5000], device='cuda:0')

In [66]:
rmse

0.838736481576567

In [57]:
predict_ratings[0][:10]

tensor([2.7650, 2.8793, 2.5193, 2.8136, 3.7364, 2.8414, 2.8343, 3.5618, 3.6456,
        3.5358], device='cuda:0')

In [58]:
real_ratings[0][:10]

tensor([3.0000, 3.0000, 3.0000, 2.5000, 3.0000, 3.0000, 3.0000, 4.5000, 3.0000,
        2.5000], device='cuda:0')